# Read in the data

In [1]:
import pandas as pd
import numpy as np
import re

data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

data = {}

for f in data_files:
    d = pd.read_csv("schools/{0}".format(f))
    data[f.replace(".csv", "")] = d

# Read in the surveys

In [2]:
all_survey = pd.read_csv("schools/survey_all.txt", delimiter="\t", encoding='windows-1252')
d75_survey = pd.read_csv("schools/survey_d75.txt", delimiter="\t", encoding='windows-1252')
survey = pd.concat([all_survey, d75_survey], axis=0)

survey["DBN"] = survey["dbn"]

survey_fields = [
    "DBN", 
    "rr_s", 
    "rr_t", 
    "rr_p", 
    "N_s", 
    "N_t", 
    "N_p", 
    "saf_p_11", 
    "com_p_11", 
    "eng_p_11", 
    "aca_p_11", 
    "saf_t_11", 
    "com_t_11", 
    "eng_t_11", 
    "aca_t_11", 
    "saf_s_11", 
    "com_s_11", 
    "eng_s_11", 
    "aca_s_11", 
    "saf_tot_11", 
    "com_tot_11", 
    "eng_tot_11", 
    "aca_tot_11",
]
survey = survey.loc[:,survey_fields]
data["survey"] = survey

# Add DBN columns

In [3]:
data["hs_directory"]["DBN"] = data["hs_directory"]["dbn"]

def pad_csd(num):
    string_representation = str(num)
    if len(string_representation) > 1:
        return string_representation
    else:
        return "0" + string_representation
    
data["class_size"]["padded_csd"] = data["class_size"]["CSD"].apply(pad_csd)
data["class_size"]["DBN"] = data["class_size"]["padded_csd"] + data["class_size"]["SCHOOL CODE"]

# Convert columns to numeric

In [4]:
cols = ['SAT Math Avg. Score', 'SAT Critical Reading Avg. Score', 'SAT Writing Avg. Score']
for c in cols:
    data["sat_results"][c] = pd.to_numeric(data["sat_results"][c], errors="coerce")

data['sat_results']['sat_score'] = data['sat_results'][cols[0]] + data['sat_results'][cols[1]] + data['sat_results'][cols[2]]

def find_lat(loc):
    coords = re.findall("\(.+, .+\)", loc)
    lat = coords[0].split(",")[0].replace("(", "")
    return lat

def find_lon(loc):
    coords = re.findall("\(.+, .+\)", loc)
    lon = coords[0].split(",")[1].replace(")", "").strip()
    return lon

data["hs_directory"]["lat"] = data["hs_directory"]["Location 1"].apply(find_lat)
data["hs_directory"]["lon"] = data["hs_directory"]["Location 1"].apply(find_lon)

data["hs_directory"]["lat"] = pd.to_numeric(data["hs_directory"]["lat"], errors="coerce")
data["hs_directory"]["lon"] = pd.to_numeric(data["hs_directory"]["lon"], errors="coerce")

# Condense datasets

In [6]:
class_size = data["class_size"]
class_size = class_size[class_size["GRADE "] == "09-12"]
class_size = class_size[class_size["PROGRAM TYPE"] == "GEN ED"]

class_size = class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace=True)
data["class_size"] = class_size

data["demographics"] = data["demographics"][data["demographics"]["schoolyear"] == 20112012]

data["graduation"] = data["graduation"][data["graduation"]["Cohort"] == "2006"]
data["graduation"] = data["graduation"][data["graduation"]["Demographic"] == "Total Cohort"]

# Convert AP scores to numeric

In [7]:
cols = ['AP Test Takers ', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']

for col in cols:
    data["ap_2010"][col] = pd.to_numeric(data["ap_2010"][col], errors="coerce")

# Combine the datasets

In [8]:
combined = data["sat_results"]

combined = combined.merge(data["ap_2010"], on="DBN", how="left")
combined = combined.merge(data["graduation"], on="DBN", how="left")

to_merge = ["class_size", "demographics", "survey", "hs_directory"]

for m in to_merge:
    combined = combined.merge(data[m], on="DBN", how="inner")

combined = combined.fillna(combined.mean())
combined = combined.fillna(0)

# Add a school district column for mapping

In [9]:
def get_first_two_chars(dbn):
    return dbn[0:2]

combined["school_dist"] = combined["DBN"].apply(get_first_two_chars)

# Find correlations

In [10]:
correlations = combined.corr()
correlations = correlations["sat_score"]
print(correlations)

SAT Critical Reading Avg. Score         0.986820
SAT Math Avg. Score                     0.972643
SAT Writing Avg. Score                  0.987771
sat_score                               1.000000
AP Test Takers                          0.523140
Total Exams Taken                       0.514333
Number of Exams with scores 3 4 or 5    0.463245
Total Cohort                            0.325144
CSD                                     0.042948
NUMBER OF STUDENTS / SEATS FILLED       0.394626
NUMBER OF SECTIONS                      0.362673
AVERAGE CLASS SIZE                      0.381014
SIZE OF SMALLEST CLASS                  0.249949
SIZE OF LARGEST CLASS                   0.314434
SCHOOLWIDE PUPIL-TEACHER RATIO               NaN
schoolyear                                   NaN
fl_percent                                   NaN
frl_percent                            -0.722225
total_enrollment                        0.367857
ell_num                                -0.153778
ell_percent         

# Plotting survey correlations

In [11]:
# Remove DBN since it's a unique identifier, not a useful numerical value for correlation.
survey_fields.remove("DBN")

In [12]:
# #Make a bar plot of the correlations between these fields and sat_score
# %matplotlib
# import matplotlib.pyplot as plt

# x = np.arange(len(survey_fields))

# fig, ax = plt.subplots()

# rects1 = ax.bar(x - 0.40, correlations)
# ax.set_ylabel('Correlation')
# ax.set_title('Correlation between SAT Scores and Different Factors')
# ax.set_xticks(x)
# ax.set_xticklabels(survey_fields)

# plt.show()
combined.corr()["sat_score"][survey_fields].plot.bar()

Consult the data dictionary that's part of the zip file you can download from the City of New York's website.

Did you find any surprising correlations?
Write up your results in a Markdown cell.

In [14]:
combined.plot.scatter("saf_s_11", "sat_score")
# plt.show()

In [ ]:
# Conclusions about safety scores and SAT scores


In [16]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

districts = combined.groupby("school_dist").agg(np.mean)
districts.reset_index(inplace=True)

m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='i'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)
# Temporary bug: if you run the following line of code in the Jupyter Guided Project interface on Dataquest, you'll get an error. 
# We're working on a fix, thanks for your patience! This should work fine locally on your own computer though.
# m.fillcontinents(color='white',lake_color='#85A6D9')

longitudes = districts["lon"].tolist()
latitudes = districts["lat"].tolist()
m.scatter(longitudes, latitudes, s=50, zorder=2, latlon=True, c=districts["saf_s_11"], cmap="summer")
plt.show()

/dataquest/system/env/python3/lib/python3.4/site-packages/plotly/matplotlylib/renderer.py:487: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

/dataquest/system/env/python3/lib/python3.4/site-packages/plotly/matplotlylib/renderer.py:451: UserWarning:

Dang! That path collection is out of this world. I totally don't know what to do with it yet! Plotly can only import path collections linked to 'data' coordinates



In [ ]:
# Write up your conclusions about safety by geographic area in a Markdown cell. You may want to read up on the boroughs of New York City.


In [17]:
race = ["white_per", "asian_per", "black_per", "hispanic_per"]
combined.corr()["sat_score"][race].plot.bar()

In [ ]:
# Write up a Markdown cell containing your findings. Are there any unexpected correlations?

In [18]:
# Explore schools with low SAT scores and high values for hispanic_per.
# Make a scatter plot of hispanic_per vs. sat_score.
combined.plot.scatter("hispanic_per", "sat_score")

In [ ]:
# What does the scatter plot show? Record any interesting observsations in a Markdown cell.


In [19]:
# Research any schools with a hispanic_per greater than 95%.
# Find the school names in the data.
# Use Wikipedia and Google to research the schools by name.
# Is there anything interesting about these particular schools? Record your findings in a Markdown cell.

combined[combined["hispanic_per"] > 95]["SCHOOL NAME"]

44                         MANHATTAN BRIDGES HIGH SCHOOL
82      WASHINGTON HEIGHTS EXPEDITIONARY LEARNING SCHOOL
89     GREGORIO LUPERON HIGH SCHOOL FOR SCIENCE AND M...
125                  ACADEMY FOR LANGUAGE AND TECHNOLOGY
141                INTERNATIONAL SCHOOL FOR LIBERAL ARTS
176     PAN AMERICAN INTERNATIONAL HIGH SCHOOL AT MONROE
253                            MULTICULTURAL HIGH SCHOOL
286               PAN AMERICAN INTERNATIONAL HIGH SCHOOL
Name: SCHOOL NAME, dtype: object

In [ ]:
# Observations


In [20]:
# Research any schools with a hispanic_per less than 10% and an average SAT score greater than 1800.
# Find the school names in the data.
# Use Wikipedia and Google to research the schools by name.
# Is there anything interesting about these particular schools? Record your findings in a Markdown cell.

combined[(combined["hispanic_per"] < 10) & (combined["sat_score"] > 1800)]["SCHOOL NAME"]

37                                STUYVESANT HIGH SCHOOL
151                         BRONX HIGH SCHOOL OF SCIENCE
187                       BROOKLYN TECHNICAL HIGH SCHOOL
327    QUEENS HIGH SCHOOL FOR THE SCIENCES AT YORK CO...
356                  STATEN ISLAND TECHNICAL HIGH SCHOOL
Name: SCHOOL NAME, dtype: object

In [ ]:
# Observations


In [21]:
# Investigate gender differences in SAT scores.
# Make a bar plot of the correlations between the columns above and sat_score.
# Record your findings in a Markdown cell. Are there any unexpected correlations?

gender = ["male_per", "female_per"]
combined.corr()["sat_score"][gender].plot.bar()

In [22]:
# Investigate schools with high SAT scores and a high female_per.
# Make a scatter plot of female_per vs. sat_score.
# What does the scatter plot show? Record any interesting observations in a Markdown cell.

combined.plot.scatter("female_per", "sat_score")


In [23]:
# Research any schools with a female_per greater than 60% and an average SAT score greater than 1700.
# Find the school names in the data.
# Use Wikipedia and Google to research the schools by name.
# Is there anything interesting about these particular schools? Record your findings in a Markdown cell.

combined[(combined["female_per"] > 60) & (combined["sat_score"])]["SCHOOL NAME"]

5                         BARD HIGH SCHOOL EARLY COLLEGE
9                  HIGH SCHOOL OF HOSPITALITY MANAGEMENT
14                          LOWER MANHATTAN ARTS ACADEMY
15     URBAN ASSEMBLY SCHOOL OF BUSINESS FOR YOUNG WO...
16                             GRAMERCY ARTS HIGH SCHOOL
22              PROFESSIONAL PERFORMING ARTS HIGH SCHOOL
26                         ELEANOR ROOSEVELT HIGH SCHOOL
27                                MILLENNIUM HIGH SCHOOL
29     HIGH SCHOOL FOR HEALTH PROFESSIONS AND HUMAN S...
34                             MANHATTAN VILLAGE ACADEMY
40                          TALENT UNLIMITED HIGH SCHOOL
43        REPERTORY COMPANY HIGH SCHOOL FOR THEATRE ARTS
45                                NEW DESIGN HIGH SCHOOL
48              RICHARD R. GREEN HIGH SCHOOL OF TEACHING
49                 THE HIGH SCHOOL OF FASHION INDUSTRIES
58     WADLEIGH SECONDARY SCHOOL FOR THE PERFORMING &...
60                                    BEACON HIGH SCHOOL
61     FIORELLO H. LAGUARDIA HI

In [24]:
combined["ap_per"] = combined["AP Test Takers "]/combined["total_enrollment"]

In [25]:
combined.plot.scatter("ap_per", "sat_score")

In [26]:
# What does the scatter plot show? Record any interesting observations in a Markdown cell.

In [27]:
# We've done quite a bit of investigation into relationships between demographics and SAT scores in this guided project. There's still quite a bit of analysis left to do, however. Here are some potential next steps:

# Determing wheter there's a correlation between class size and SAT scores
# Figuring out which neighborhoods have the best schools
# If we combine this information with a dataset containing property values, we could find the least expensive neighborhoods that have good schools.
# Investigating the differences between parent, teacher, and student responses to surveys.
# Assigning scores to schools based on sat_score and other attributes.
# We recommend creating a GitHub repository and placing this project there. It will help other people see your work, including employers. As you start to put multiple projects on GitHub, you'll have the beginnings of a strong portfolio.

# You're welcome to keep working on the project here, but we recommend downloading it to your computer using the download icon above and working on it there.

# Curious to see what other students have done on this project? Head over to our Community to check them out. While you are there, please remember to show some love and give your own feedback!

# And of course, we welcome you to share your own project and show off your hard work. Head over to our Community to share your finished Guided Project!



In [ ]:
Go through all thes